In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import shutil
import random
import xml.etree.ElementTree as ET
from glob import glob

random.seed(42)

# Constants
CARDS = [f"{c}{s}" for c in ['2','3','4','5','6','7','8','9','10','J','Q','K','A'] for s in ['C', 'S', 'D', 'H']] + ["JOKER"]
CHIP_DIR = "/content/drive/MyDrive/CV_blackjack/DEGEN101.v16i.voc"
CARD_IMG_DIR = '/content/drive/MyDrive/CV_blackjack/archive/Images/Images'
CARD_XML_DIR = '/content/drive/MyDrive/CV_blackjack/archive/Annotations/Annotations'
YOLO_BASE = "/content/drive/MyDrive/CV_blackjack/yolo_dataset"
SPLITS = ['train', 'val']
CLASSES = CARDS + ["Chip"]
CLASS_IDX = {cls_name: idx for idx, cls_name in enumerate(CLASSES)}

# creatin space where data would be stored
if os.path.exists(YOLO_BASE):
    shutil.rmtree(YOLO_BASE)
for split in SPLITS:
    os.makedirs(os.path.join(YOLO_BASE, "images", split), exist_ok=True)
    os.makedirs(os.path.join(YOLO_BASE, "labels", split), exist_ok=True)

# Convert VOC to YOLO
def convert_bbox(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[2]) / 2.0
    y = (box[1] + box[3]) / 2.0
    w = box[2] - box[0]
    h = box[3] - box[1]
    return (x * dw, y * dh, w * dw, h * dh)

def process_and_save(img_path, xml_path, split):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    filename = os.path.basename(img_path)
    dst_img = os.path.join(YOLO_BASE, "images", split, filename)
    dst_label = os.path.join(YOLO_BASE, "labels", split, os.path.splitext(filename)[0] + ".txt")

    # copy image
    shutil.copy(img_path, dst_img)

    # extract label and bbox
    size = root.find("size")
    w, h = int(size.find("width").text), int(size.find("height").text)
    obj = root.find("object")
    label = obj.find("name").text
    if label not in CLASS_IDX:
        print(f"Label {label} not recognized")
        return

    bbox = obj.find("bndbox")
    box = [int(bbox.find(t).text) for t in ["xmin", "ymin", "xmax", "ymax"]]
    bb = convert_bbox((w, h), box)

    # write YOLO label
    with open(dst_label, "w") as f:
        f.write(f"{CLASS_IDX[label]} {' '.join([str(round(x, 6)) for x in bb])}\n")

In [ ]:
# Process cards
for card in CARDS:
    indices = list(range(51))
    random.shuffle(indices)
    train_idx, val_idx = indices[:46], indices[46:]

    for split, idx_list in zip(SPLITS, [train_idx, val_idx]):
        for idx in idx_list:
            img_name = f"{card}{idx}.jpg"
            xml_name = f"{card}{idx}.xml"
            img_path = os.path.join(CARD_IMG_DIR, img_name)
            xml_path = os.path.join(CARD_XML_DIR, xml_name)

            if os.path.exists(img_path) and os.path.exists(xml_path):
                process_and_save(img_path, xml_path, split)
            else:
                print(f"Missing: {img_path} or {xml_path}")

In [ ]:
# Process chips (already split)
for split in SPLITS+["test"]:
  chip_dir = os.path.join(CHIP_DIR, split)
  xml_files = glob(os.path.join(chip_dir, "*.xml"))

  for xml_path in xml_files:
    img_path = xml_path.replace(".xml", ".jpg")
    if os.path.exists(img_path) and split in ["train", "test"]:
      print(split)
      process_and_save(img_path, xml_path, "train")
    elif os.path.exists(img_path):
      process_and_save(img_path,xml_path,"val")
    else:
      print(f"Missing chip image: {img_path}")


/content/drive/MyDrive/CV_blackjack/DEGEN101.v16i.voc/train/IMG_3775_jpeg.rf.89d74495049c7dd2de900e9d27c06aa8.xml
/content/drive/MyDrive/CV_blackjack/DEGEN101.v16i.voc/train/IMG_3820_jpeg.rf.335dc7c0f331a972ad7fbaade1c94b7d.xml
/content/drive/MyDrive/CV_blackjack/DEGEN101.v16i.voc/train/20230503_000805_jpg.rf.316a27e9e8c093dbc8fac5d9be57c10d.xml
/content/drive/MyDrive/CV_blackjack/DEGEN101.v16i.voc/train/IMG_0456_jpeg.rf.79e5f7cef4df538d88beb0183720ff7b.xml
/content/drive/MyDrive/CV_blackjack/DEGEN101.v16i.voc/train/IMG_3864_jpeg.rf.5177d5f59338cd1dd88ea940e4a3bb10.xml
/content/drive/MyDrive/CV_blackjack/DEGEN101.v16i.voc/train/IMG_0454_jpeg.rf.a5f2a6334a00ef5abf00bb26fb9b7a0f.xml
/content/drive/MyDrive/CV_blackjack/DEGEN101.v16i.voc/train/20230502_235417_jpg.rf.a1f6a18bc6afd1a50ff5b39919b1ab82.xml
/content/drive/MyDrive/CV_blackjack/DEGEN101.v16i.voc/train/20230503_000418_jpg.rf.5bfc93fa98aa2d4e95000126fe2d6c77.xml
/content/drive/MyDrive/CV_blackjack/DEGEN101.v16i.voc/train/IMG_3874_j

In [ ]:
yaml_content = f"""
train: /content/drive/MyDrive/CV_blackjack/yolo_dataset/images/train
val: /content/drive/MyDrive/CV_blackjack/yolo_dataset/images/val

nc: 54

names: {CLASSES}
"""

with open(os.path.join("/content/drive/MyDrive/CV_blackjack/yolo_dataset/data.yaml"), 'w') as file:
    file.write(yaml_content)

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 984.0/984.0 kB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8s.pt')

model.train(
    data='/content/drive/MyDrive/CV_blackjack/yolo_dataset/data.yaml',
    imgsz=416,
    epochs=30,
    batch=16,
    name='blackjack-v8s',
    half=True,
    workers=2,
    device=0,
    hsv_h= 0.0,
    hsv_s= 0.0,
    hsv_v= 0.0,
    flipud= 0.5,
    fliplr= 0.5,
    scale= 0.5,
    degrees= 10.0,
    translate= 0.1,
    shear= 2.0,
    perspective= 0.0,
    copy_paste= 0.0,
    mixup= 0.0,
    mosaic=1.0
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 21.5M/21.5M [00:00<00:00, 391MB/s]


Ultralytics 8.3.116 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/CV_blackjack/yolo_dataset/data.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=0, workers=2, project=None, name=blackjack-v8s, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=True, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_con

100%|██████████| 755k/755k [00:00<00:00, 105MB/s]

Overriding model.yaml nc=80 with nc=54

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytic

...

                   all        422        422      0.586      0.878      0.771      0.486



30 epochs completed in 1.974 hours.
Optimizer stripped from runs/detect/blackjack-v8s/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/blackjack-v8s/weights/best.pt, 22.5MB

Validating runs/detect/blackjack-v8s/weights/best.pt...
Ultralytics 8.3.116 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,146,482 parameters, 0 gradients, 28.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:22<00:00,  1.59s/it]


                   all        422        422      0.593      0.881      0.782      0.494
                    2C          5          5      0.495        0.6      0.606      0.299
                    2S          5          5      0.591          1      0.928      0.603
                    2D          5          5      0.878          1      0.995      0.589
                    2H          5          5      0.499          1      0.622      0.437
                    3C          5          5      0.612        0.8      0.706      0.501
                    3S          5          5      0.611          1      0.711      0.559
                    3D          5          5      0.517      0.863      0.747      0.367
                    3H          5          5      0.649          1      0.881      0.557
                    4C          5          5      0.494          1      0.866      0.597
                    4S          5          5      0.421        0.6      0.479       0.24
                    4

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x79f952ccbd10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,   

In [ ]:
model = YOLO('/content/drive/MyDrive/CV_blackjack/best_model.pt')
metrics = model.val()

Ultralytics 8.3.116 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,016,178 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.5±0.1 ms, read: 105.7±83.0 MB/s, size: 419.7 KB)


val: Scanning /content/drive/.shortcut-targets-by-id/1JUnZ0fD6yU4vv7R7FMdfaaNJ55SavGrG/CV_blackjack/yolo_dataset/labels/val.cache... 422 images, 0 backgrounds, 0 corrupt: 100%|██████████| 422/422 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 27/27 [00:31<00:00,  1.16s/it]


                   all        422        422      0.582      0.796      0.747      0.466
                    2C          5          5      0.702          1      0.732      0.336
                    2S          5          5      0.697          1      0.995       0.67
                    2D          5          5      0.855          1      0.995       0.53
                    2H          5          5      0.496        0.8      0.586       0.39
                    3C          5          5      0.541        0.6      0.616       0.38
                    3S          5          5      0.686          1      0.779      0.581
                    3D          5          5      0.546        0.6      0.695      0.366
                    3H          5          5      0.615          1      0.831      0.508
                    4C          5          5      0.518        0.8      0.855      0.553
                    4S          5          5      0.347        0.4      0.497       0.29
                    4

In [1]:
import cv2
video_path = '/content/drive/MyDrive/CV_blackjack/IMG_0636.mp4'
cap = cv2.VideoCapture(video_path)

# Output setup
out_path = '/content/drive/MyDrive/CV_blackjack/output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(out_path, fourcc, fps, (width, height))
# delete later
print("FPS:", fps)
print("Width:", width)
print("Height:", height)
print("Out Path:", out_path)
print("Codec:", fourcc)
print(f"Saving to: {out_path}")
if not out.isOpened():
    print("VideoWriter failed to open.")

FPS: 0.0
Width: 0
Height: 0
Out Path: /content/drive/MyDrive/CV_blackjack/output_video.mp4
Codec: 1983148141
Saving to: /content/drive/MyDrive/CV_blackjack/output_video.mp4
VideoWriter failed to open.


In [4]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 984.0/984.0 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 33.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [5]:
from ultralytics import YOLO

model = YOLO('/content/drive/MyDrive/CV_blackjack/best_model.pt')
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Inference with your model
    results = model.predict(source=frame, save=False, conf=0.3)

    # Draw boxes
    annotated_frame = results[0].plot()

    # Write frame
    out.write(annotated_frame)

cap.release()
out.release()

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/CV_blackjack/best_model.pt'

In [8]:
!git config --global user.email "rk3230@columbia.edu"
!git config --global user.name "RaiymbekKokteubay"

In [11]:
from getpass import getpass
token = getpass('Enter your GitHub token: ')

!git clone https://{token}@github.com/RaiymbekKokteubay/poker_cards-chips_detection.git

Enter your GitHub token: ··········
Cloning into 'poker_cards-chips_detection'...
remote: Enumerating objects: 10905, done.
remote: Total 10905 (delta 0), reused 0 (delta 0), pack-reused 10905 (from 1)
Receiving objects: 100% (10905/10905), 1.63 GiB | 24.11 MiB/s, done.
Resolving deltas: 100% (3940/3940), done.
Updating files: 100% (10893/10893), done.


In [ ]:
!cp -r /content/drive/MyDrive/CV_blackjack/* /content/poker_cards-chips_detection/

In [ ]:
%cd /content/poker_cards-chips_detection/
!git add .
!git commit -m "model development and training"
!git push